In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd
import spacy


# Import Required Libraries & Data Loading

In [2]:
#importing the training data
df=pd.read_csv('../input/imdb-dataset/IMDB Dataset.csv')
print(df.shape)
df.head(10)

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
5,"Probably my all-time favorite movie, a story o...",positive
6,I sure would like to see a resurrection of a u...,positive
7,"This show was an amazing, fresh & innovative i...",negative
8,Encouraged by the positive comments about this...,negative
9,If you like original gut wrenching laughter yo...,positive


# Data Preparation

In [3]:
"""
sentiment : 0 = negative, 1 = positive 
use the following to get the sentiment of a sentence :  
sentiment = 0 if sentiment is negative else 1


use np.where to get the sentiment of a sentence :
"""
df['sentiment'] = np.where(df['sentiment'] == 'positive', 1, 0)

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [5]:
df.columns = ['TEXT_COLUMN_NAME', 'LABEL_COLUMN_NAME']

In [6]:
"""
Load the spacy model and load the English language model from https://spacy.io/usage/models
"""
spacy.load('en_core_web_sm')### ADD YOUR SPACY MODEL HERE ###

# Text & label Preparation

In [7]:
# Define feature processing
"""
Define the fields for the data.
"""
TEXT = torchtext.legacy.data.Field(tokenize = 'spacy', tokenizer_language = 'en_core_web_sm')

In [8]:
# Define Label processing
LABEL = torchtext.legacy.data.LabelField(dtype = torch.long)

In [9]:
"""
Define the fields for the data.
"""

df.to_csv('/tmp/moviedata.csv', index = None)
df = pd.read_csv('/tmp/moviedata.csv')
df.head()

,TEXT_COLUMN_NAME,LABEL_COLUMN_NAME
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
# process the dataset

field = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]
try: 
  dataset = torchtext.legacy.data.TabularDataset(
                    path = '/tmp/moviedata.csv', ### ADD YOUR DATASET PATH HERE ###
                    format = 'csv', ### ADD YOUR DATASET FORMAT HERE ###
                    skip_header ='True' , ### ADD YOUR SKIP HEADER HERE ### 
                    fields =  field### ADD YOUR FIELDS HERE ### 
  )
except Exception as e:
  print(e)

In [11]:
vars(dataset.examples[0])

{'TEXT_COLUMN_NAME': ['One',
  'of',
  'the',
  'other',
  'reviewers',
  'has',
  'mentioned',
  'that',
  'after',
  'watching',
  'just',
  '1',
  'Oz',
  'episode',
  'you',
  "'ll",
  'be',
  'hooked',
  '.',
  'They',
  'are',
  'right',
  ',',
  'as',
  'this',
  'is',
  'exactly',
  'what',
  'happened',
  'with',
  'me.<br',
  '/><br',
  '/>The',
  'first',
  'thing',
  'that',
  'struck',
  'me',
  'about',
  'Oz',
  'was',
  'its',
  'brutality',
  'and',
  'unflinching',
  'scenes',
  'of',
  'violence',
  ',',
  'which',
  'set',
  'in',
  'right',
  'from',
  'the',
  'word',
  'GO',
  '.',
  'Trust',
  'me',
  ',',
  'this',
  'is',
  'not',
  'a',
  'show',
  'for',
  'the',
  'faint',
  'hearted',
  'or',
  'timid',
  '.',
  'This',
  'show',
  'pulls',
  'no',
  'punches',
  'with',
  'regards',
  'to',
  'drugs',
  ',',
  'sex',
  'or',
  'violence',
  '.',
  'Its',
  'is',
  'hardcore',
  ',',
  'in',
  'the',
  'classic',
  'use',
  'of',
  'the',
  'word.<br',
  '

# Data Split

In [12]:
# Split dataset into train and test set
RANDOM_SEED = 123
train_data, test_data = dataset.split(split_ratio = [0.8, 0.2], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of test data', len(test_data))

Length of train data 40000
Length of test data 10000


In [13]:
train_data, val_data = train_data.split(split_ratio = [0.85, 0.15], random_state = random.seed(RANDOM_SEED))

print('Length of train data', len(train_data))
print('Length of valid data', len(val_data))

Length of train data 34000
Length of valid data 6000


# Data Observation after Tokenization

In [14]:
# Look at first traning example

print(vars(train_data.examples[2000]))

{'TEXT_COLUMN_NAME': ['Flipping', 'through', 'the', 'channels', 'I', 'was', 'lucky', 'enough', 'to', 'stumble', 'upon', 'the', 'beginning', 'of', 'this', 'movie', '.', 'I', 'must', 'admit', 'that', 'it', 'grabbed', 'my', 'attention', 'almost', 'immediately', '.', 'I', 'love', 'older', 'films', 'and', 'this', 'is', 'or', 'should', 'be', 'considered', 'a', 'classic', '!', 'One', 'of', 'the', 'most', 'wonderful', 'rarities', 'of', 'this', 'movie', 'is', 'that', 'the', 'main', 'character', 'was', 'not', 'only', 'female', 'but', 'she', 'was', 'also', 'a', 'bad', 'girl', '.', 'I', 'highly', 'recommend', 'this', 'movie', '!'], 'LABEL_COLUMN_NAME': '1'}


In [15]:
# Build Vocabulary
VOCABULARY_SIZE = 20000
TEXT.build_vocab(train_data, max_size = VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'vocabulary size: {len(TEXT.vocab)}')
print(f'Label Size: {len(LABEL.vocab)}')

vocabulary size: 20002
Label Size: 2


In [16]:
a= vars(TEXT.vocab)
x= a['freqs']
sorted_freqs=[(l,k) for k,l in sorted([(j,i) for i,j in x.items()], reverse=True)]

 2 extra value in vocabulary is because added (unknown) and (padding)

In [17]:
# Print the most common words: Use the most_common method of the TEXT vocabulary
most_common_words = sorted_freqs[0:20]
print(most_common_words)

[('the', 390972), (',', 369444), ('.', 318509), ('a', 210502), ('and', 210008), ('of', 194659), ('to', 180163), ('is', 145895), ('in', 118266), ('I', 105681), ('it', 103588), ('that', 93995), ('"', 85535), ("'s", 83149), ('this', 81775), ('-', 71249), ('/><br', 68787), ('was', 67372), ('as', 57734), ('movie', 57572)]


In [18]:
# Token corresponding to first 10 Indices

print(TEXT.vocab.itos[:20]) #itos = Integer to string

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


# Data Preparation for Batch wise Implimentation

In [19]:
# Define Dataloader
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_loader, valid_loader, test_loader = torchtext.legacy.data.BucketIterator.splits(
        (train_data, val_data, test_data), ### ADD YOUR SPLIT DATA HERE (Make sure you add it in a tuple) ###
        batch_size = 128, ### ADD YOUR BATCH SIZE HERE ###
        sort_within_batch = True, ### ADD YOUR SORT WITHIN BATCH HERE ### 
        sort_key = lambda x : len(x.TEXT_COLUMN_NAME), 
        device = DEVICE
    )

In [20]:
# Testing the iterators (note that the number of rows depends on the longest document in the respective batch):

print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([250, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([56, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([50, 128])
Target vector size: torch.Size([128])


# Model Building

In [50]:
from torch import nn
class RNN(nn.Module):
  def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
    super().__init__() #to call the functions in the superclass
    self.embedding = nn.Embedding(input_dim, embedding_dim) #Embedding layer to create dense vector instead of sparse matrix
    self.rnn = nn.RNN(embedding_dim, hidden_dim) 
    self.hidden_fc = nn.Linear(hidden_dim,hidden_dim)
    self.out_fc = nn.Linear(hidden_dim, output_dim)
    self.dropout = nn.Dropout(0.3)
  def forward(self, text):
    embedded = self.embedding(text)
    output, hidden = self.rnn(embedded)   
    hidden = self.dropout(hidden[-1,:,:])
    hidden = F.relu(self.hidden_fc(hidden))
    return self.out_fc(hidden)

In [76]:
class RNN1(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        ### ADD YOUR CODE HERE ###
        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = torch.nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        # embedding and LSTM layers
        
        ### END YOUR CODE ### 

    def forward(self, text):
        ### ADD YOUR CODE HERE ###
        # text dim: [sentence length, batch size]
        # embedded dim: [sentence length, batch size, embedding dim]
        embedded = self.embedding(text)
        output, hidden = self.rnn(embedded)   
        
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        # hidden dim: [batch size, hidden dim]
        assert torch.equal(output[-1,:,:],hidden.squeeze(0))
        
        ### END YOUR CODE ###
        #output = 
        return self.fc(hidden.squeeze(0))

In [83]:
torch.manual_seed(RANDOM_SEED)

model = RNN1(input_dim= len(TEXT.vocab), ### ADD YOUR INPUT DIM HERE. This can be the length of your vocabulary or the embedding dim ###
            embedding_dim=400, ### ADD YOUR EMBEDDING DIM HERE ###
            hidden_dim=128, ### ADD YOUR HIDDEN DIM HERE ###
            output_dim=2  ### ADD NUMBER OF CLASSES HERE ###
)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(DEVICE)
optimizer =  torch.optim.Adam(model.parameters(), lr=1e-3)### ADD YOUR OPTIMIZER HERE ###

# Define Accuracy

In [23]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

# Model Run

In [87]:
start_time = time.time()
NUM_EPOCHS = 50
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
#criterion = torch.nn.BCEWithLogitsLoss()
model = model.to(DEVICE)
criterion = criterion.to(DEVICE)
for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
                
        predictions = model(text)
        
        loss = F.cross_entropy(predictions, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        ### UPDATE MODEL PARAMETERS
        
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Loss: {loss:.4f}')     
    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/050 | Batch 000/266 | Loss: 0.3095
Epoch: 001/050 | Batch 050/266 | Loss: 0.3761
Epoch: 001/050 | Batch 100/266 | Loss: 0.5528
Epoch: 001/050 | Batch 150/266 | Loss: 0.4029
Epoch: 001/050 | Batch 200/266 | Loss: 0.5393
Epoch: 001/050 | Batch 250/266 | Loss: 0.7313
training accuracy: 62.65%
valid accuracy: 57.98%
Time elapsed: 0.18 min
Epoch: 002/050 | Batch 000/266 | Loss: 0.7223
Epoch: 002/050 | Batch 050/266 | Loss: 0.6451
Epoch: 002/050 | Batch 100/266 | Loss: 0.5268
Epoch: 002/050 | Batch 150/266 | Loss: 0.5357
Epoch: 002/050 | Batch 200/266 | Loss: 0.5202
Epoch: 002/050 | Batch 250/266 | Loss: 0.3992
training accuracy: 80.88%
valid accuracy: 70.68%
Time elapsed: 0.35 min
Epoch: 003/050 | Batch 000/266 | Loss: 0.4783
Epoch: 003/050 | Batch 050/266 | Loss: 0.3261
Epoch: 003/050 | Batch 100/266 | Loss: 0.4176
Epoch: 003/050 | Batch 150/266 | Loss: 0.6314
Epoch: 003/050 | Batch 200/266 | Loss: 0.9468
Epoch: 003/050 | Batch 250/266 | Loss: 0.5817
training accuracy: 74.89%
va

# Model Testing

In [90]:
import spacy


nlp = spacy.blank("en")

def predict_sentiment(model, sentence):

    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.nn.functional.softmax(model(tensor), dim=1)
    return prediction[0][1].item()

print('Probability positive:')
predict_sentiment(model, "This is such an awesome movie, I really love it!")

Probability positive:


0.9224046468734741

In [91]:
print('Probability positive:')
predict_sentiment(model, "I really hate this movie. It is really bad and sucks!")

Probability positive:


0.01575806923210621